* Add (새 글 등록)과 Edit (수정) 기능 합치기

등록과 수정을 위한 V,T 코드를 작성하고 나니, 두 기능 모두 같은 형태의 Form을 공유하고 있다는 것을 알 수 있다. 따라서 이를 하나로 결합하고자 한다.

구체적으로 pk=0 일때는 add 페이지로, 그 외에는 edit 페이지로 설계한다. 이러한 구조가 가능한 이유는, DB에 primary key가 0인 데이터는 존재할 수 없기 때문이다(?)

또, edit.html을 jQuery (java script)를 이용하여 더 예쁘게 꾸미려고 한다. jQuery 및 java script에 관한 내용은 다른 곳에서 다룬다. 아니면 알아서 찾아보셈.

```python
# /blgo/views.py 파일
# class PostView
class PostEditView(View):
    def get(self, request, pk):
        if pk == 0:
            form = PostForm()
        else:
            post = get_object_or_404(Post, pk=pk)
            form = PostForm(initial={'title':post.title, 'text':post.text})
        return render(request, 'blog/edit.html', {'form':form})
    
    def post(self, request, pk):
        form = PostForm(request.POST)
        if form.is_valid():
            post = get_object_or_404(Post, pk=pk)
            post.title, post.text = form['title'].value(), form['text'].value()
            post.publish()
            return redirect('/blog/Blist')
        return render(request, 'blog/edit.html', {'form':form, 'pk':pk})
```

.../0/edit 으로 접속 시 새 글 등록하기 위한 페이지가 보여준다. 이를 위해 비어있는 form을 제공한다.

..../18/edit 으로 접근 시 Post 테이블의 pk 값이 18번인 데이터(포스트)를 수정하는 것을 의미한다.

```html
<!-- ch99/templates/blog/edit.html 파일 -->
<!-- jQuery를 사용하기 위한 모듈 import -->
<script src="http://code.jquery.com/jquery-1.11.3.min.js"></script>
<script src="http://code.jquery.com/jquery-migrate-1.2.1.min.js"></script>

<!-- 제목(title) form의 배경을 위한 style -->
<style>
    .bg { bakcground-color: #eeeee; }
    .bd { border: 1px solid #66666; }
</style>

{% if form.title.value %}
수정하기
{% else %}
신규작성
{% endif %}

<form method='POST'>
    {% csrf_token %}
    {{form.as_p}}
    <input type=submit value='작성'> <!-- 수정에서 작성으로 변경 -->
</form>

<script>
    $("#id_title").addClass('bg bd');
</script>
```

form 태그에서 action을 없애면 이전에 코딩했던 `action='.'`과 같이 자기 자신을 호출한다.

* Form과 Model 결합

지금까지는 form과 model을 개별적인 객체로 취급하여 코딩했다. 그래서 views.py 파일에 일일이 form과 model을 매핑시켜주는 코드를 작성하였다.

이번에는 form과 model을 연동하여 코드를 작성한다. 이를 위해 ModelForm을 이용한다. 이렇게 작성하면 일일이 form tag를 만들 필요가 없으며, 개발자가 필요한 field (Post 테이블의 하나의 column) 값만 갖고 와서 사용할 수 있다.

그리고 일반적으로 Form 관련 사항은 forms.py 파일을 생성하여 그 안에서 따로 관리한다. 이번 실습도 form 관련 내용을 forms.py로 분리하는 것까지 진행한다.

주의사항으로 Django의 Model을 사용해서 DB를 구성한 경우(models.py->migrate)에만 ModelForm을 사용할 수 있다. 즉, 이미 DB가 생성되어 있는 경우 앞서 코딩한 것과 같은 방식으로 개발해야 한다.

**근데 솔직히 이 과정이 어떻게 이뤄졌는지 잘 모르겠으니 그냥 그려러니 하고 넘어가라**

```python
# /blog/views.py 파일
from django.shortcuts import render, get_object_or_404, redirect
from django.http import HttpResponse
from django.views.generic import View
from django.contrib.auth import authenticate
from django.contrib.auth.models import User
# from django.forms import Form, CharField, Textarea
# from django.core.exceptions import ValidationError
# from django import forms

from blog.models import Post
from . import forms

class PostEditView(View):
    def get(self, request, pk):
        if pk == 0:
            form = forms.PostForm()
        else:
            post = get_object_or_404(Post, pk=pk)
            form = forms.PostForm(instance=post)
        return render(request, "blog/edit.html", {"form":form})

    def post(self, request, pk):
        username = request.session["username"]
        user = User.objects.get(username=username)

        if pk == 0:
            form = forms.PostForm(request.POST)
        else:
            post = get_object_or_404(Post, pk=pk)
            form = forms.PostForm(request.POST, instance=post)

        if form.is_valid():
            post = form.save(commit=False)
            if pk == 0:
                post.author = user
                post.save()
            else:
                post.publish()
            return redirect("list")
        return render(request, "blog/edit.html", {"form": form})
```

```python
# /blog/forms.py 파일
from django import forms
from django.forms import ValidationError
from . import models

def TitleValidator(value):
    if len(value) < 5:
        raise ValidationError('제목이 너무 짧습니다.')

class PostForm(forms.ModelForm):
    class Meta:
        model = models.Post
        fields = ['title', 'text']
    
    def __init__(self, *args, **kwargs):
        super(PostForm, self).__init__(*args, **kwargs)
        self.fields['title'].validator=[TitleValidator]
```

# URL 정리하기

URL의 일관성을 위해 지금까지 난잡하게 생성했던 URL을 정리한다. 이를 위해 다음과 같은 기준을 세웠다.

1. 전체 포스트에 대한 리스트를 출력할 때: .../blog/0/list
2. 하나의 포스트를 볼 때: .../blog/18/detail
3. 새로운 포스트 작성: .../blog/0/add -> 처음 접속 후 버튼 누르면 POST 방식 접근
4. 포스트를 수정할 때: .../blog/18/edit -> 처음 접속 후 버튼 누르면 POST 방식 접근

위의 규칙들을 URL로 표현하면 아래와 같다.

`.../blog/<int:pk>/<mode>`: `<mode>` 부분에 list/detail/add/edit 중에 하나가 들어가게 된다.

```python
# /blog/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('', views.index),
    path('login/', views.LoginView.as_view()),
    path('<int:pk>/<mode>/', views.PostEditView.as_view(), name="edit"),
]
```

```python
# /blog/views.py
from django.shortcuts import render, get_object_or_404, redirect
from django.http import HttpResponse
from django.views.generic import View
from django.contrib.auth import authenticate
from django.contrib.auth.models import User

from blog.models import Post
from . import forms

# Create your views here.

def index(req):
    return HttpResponse('이번에는 제발 되게 해주세요...')

class LoginView(View):
    def get(self, request):
        return render(request, 'blog/login.html')
    
    def post(self, request):
        un = request.POST.get('username')
        pw = request.POST.get('password')
        
        user = authenticate(username=un, password=pw)
        
        if user == None:
            return redirect('/blog/login')
        else:
            request.session['username'] = un
            return redirect('/blog/0/list')

class PostEditView(View):
    def get(self, request, pk, mode):
        if mode == 'list': # mode가 list일 때. pk 값은 상관이 없다.
            username = request.session.get('username', 'home')
            user = User.objects.get(username=username)
            data = Post.objects.all().filter(author=user) # author field로 검색 후 반환
            context = {'username':user, 'data':data} # html에서 읽기 위해 json화(=딕셔너리화) 시킴.
            return render(request, 'blog/list.html', context)
        elif mode == 'detail':
            post = get_object_or_404(Post, pk=pk) # DB내 Post table에서 요청된 pk 값과 일치하는 포스트 반환
            form = forms.PostForm(instance=post) # instance=post는 무엇을 위한 것인지 모르겠음. 아마 초기 값(원래 작성된 제목 및 본문)을 위한 것 같음.
            return render(request, 'blog/detail.html', {'form':form, 'pk':pk}) # detail.html에서 수정을 위해 pk 값을 넘겨주어야함. 따라서 pk 값을 전달인자에 포함시킴
        elif mode == 'add':
            form = forms.PostForm()
            return render(request, 'blog/edit.html', {'form':form})
        elif mode == 'edit': # 여기서 return이 없는데 이는 맨 밑에 return이 존재하기 때문이다. 즉 아래 edit으로 처음 접속하면(GET 방식으로) 아래 두 줄 수행 후 맨 밑의 return 코드가 실행된다.
            post = get_object_or_404(Post, pk=pk)
            form = forms.PostForm(instance=post)
        else: return HttpResponse('무얼 하고 싶은겁니까 hUMan') # mode에서 정의되지 않은 주소로 접속하면 보여줄 메세지.
        
        return render(request, 'blog/edit.html', {'form':form})
        
    def post(self, request, pk, mode):
        username = request.session["username"]
        user = User.objects.get(username=username)

        if pk == 0: # POST 방식으로 접근했을 때 pk가 0인 경우는 add 밖에 없다(?)
            form = forms.PostForm(request.POST)
        else:
            post = get_object_or_404(Post, pk=pk)
            form = forms.PostForm(request.POST, instance=post)

        if form.is_valid():
            post = form.save(commit=False)
            if pk == 0:
                post.author = user
                post.save()
            else:
                post.publish()
            return redirect("edit",0,"list")
        return render(request, "blog/edit.html", {"form": form})
```

```html
<!-- ch99/templates/blog/list.html -->
{% extends 'blog/base.html' %}

{% block content %}

    {{username}}님 환영합니다.<br>

    <h2> {{username}}'s Posts </h2>
    
    <h3><a href="{% url 'edit' 0 'add' %}">새 글 등록</a></h3><br>

    {% for d in data %}
        <a href="{% url 'edit' d.pk 'detail' %}">{{d.title}}</a><br>
    {% endfor %}

{% endblock %}
```

```html
<!-- ch99/templates/blog/detail.html -->
{% extends 'blog/base.html' %}

{% block content %}

    <h2>Detail post</h2><br>

    {{form.as_p}}

    <a href="{% url 'edit' pk 'edit' %}">수정</a>

{% endblock%}
```

```html
<!-- ch99/templates/blog/edit.html -->
<script src="http://code.jquery.com/jquery-1.11.3.min.js"></script>
<script src="http://code.jquery.com/jquery-migrate-1.2.1.min.js"></script>

<style>
    .bg { bakcground-color: #b06363; }
    .bd { border: 1px solid #A4E519; }
</style>

{% if form.title.value %}
<h3>수정하기</h3>
{% else %}
<h3>신규작성</h3>
{% endif %}

<form method='POST'>
    {% csrf_token %}
    {{form.as_p}}
    <input type=submit value='작성'>
</form>

<script>
    $("#id_title").addClass('bg bd');
</script>
```

# 게시판 만들기

